In [1]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('data/train.csv', index_col='Id')
test = pd.read_csv('data/test.csv', index_col='Id')

In [4]:
train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [7]:
print(train.shape)
test.shape

(1460, 80)


(1459, 79)

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

<b>Nếu feature nào mất dữ liệu lớn hơn 20% toàn bộ dữ liệu thì drop feature đó </b>

In [12]:
def filt(dataset):
    n = len(dataset)
    missing_cols = []

    for col in dataset.columns:
        if dataset[col].isna().sum() >= int(0.2*n):
            missing_cols.append(col)

    print(missing_cols, "features removed from dataset")
    dataset.drop(missing_cols, axis=1, inplace=True)
    return dataset

train = filt(train)
test = filt(test)

['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'] features removed from dataset
['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'] features removed from dataset


In [15]:
missing_data_count = train.isna().sum()
missing_data_percent = missing_data_count / len(train) * 100
missing_data = pd.DataFrame({
    'Count': missing_data_count,
    'Percent': missing_data_percent
})
missing_data = missing_data[missing_data.Count > 0]
missing_data.sort_values(by='Count', ascending=False, inplace=True)
print('There are {} features that have missing values'.format(len(missing_data.index)))
missing_data.head(34)

There are 14 features that have missing values


,Count,Percent
LotFrontage,259,17.739726
GarageType,81,5.547945
GarageYrBlt,81,5.547945
GarageFinish,81,5.547945
GarageQual,81,5.547945
GarageCond,81,5.547945
BsmtExposure,38,2.602740
BsmtFinType2,38,2.602740
BsmtQual,37,2.534247
BsmtCond,37,2.534247


In [16]:
def filt_(dataset):
    for col in dataset.columns:
        if dataset[col].dtypes == 'object':
            dataset[col].fillna(dataset[col].mode()[0], inplace=True)
        elif dataset[col].dtypes == 'int64' or dataset[col].dtypes == 'float64':
            dataset[col].fillna(dataset[col].mean(), inplace=True)

    return dataset

train = filt_(train)
test = filt_(test)

In [19]:
print(train.isna().sum().sum())
test.isna().sum().sum()

0


0

In [26]:
all_data = pd.concat([train.drop(['SalePrice'], axis=1), test], axis=0)
categorical_features = all_data.select_dtypes(include='object').columns

In [ ]:
def category_onehot_multcols(multcolumns):
    df_final = all_data
    i = 0
    for field in multcolumns:
        print(field)
        df1 = pd.get_dummies(all_data[field],drop_first=True)
        
        all_data.drop([field],axis=1,inplace=True)
        if i == 0:
            df_final = df1.copy()
        else:
            
            df_final = pd.concat([df_final,df1],axis=1)
        i = i+1
       
        
    df_final = pd.concat([all_data,df_final],axis=1)
        
    return df_final